In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import rawpy
import cv2
import math

## Resizing images to 500px the long edge by interpolation

In [ ]:
def long_edge_resize500(img):
    height = img.shape[0]
    width = img.shape[1]
    if height>width:
        new_h = 500
        new_w = int(width/height*500) 
    else:
        new_w=500
        new_h=int(height/width*500)
    print("new shape is {},{}".format(new_w, new_h))
    return cv2.resize(img, (new_w, new_h), cv2.INTER_AREA)
        
    

## Conversion from RGB to LAB and only isolate L

In [ ]:
raw_RGB = rawpy.imread("data/a0001-jmac_DSC1459.dng").postprocess()
expertA_RGB = cv2.cvtColor(cv2.imread("/media/samuel/HDD1TB/data.csail.mit.edu/graphics/fivek/img/tiff16_a/a0001-jmac_DSC1459.tif", cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
#expertB_RGB = cv2.cvtColor(cv2.imread("/media/samuel/HDD1TB/data.csail.mit.edu/graphics/fivek/img/tiff16_b/a0001-jmac_DSC1459.tif", cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
#expertC_RGB = cv2.cvtColor(cv2.imread("/media/samuel/HDD1TB/data.csail.mit.edu/graphics/fivek/img/tiff16_c/a0001-jmac_DSC1459.tif", cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
#expertD_RGB = cv2.cvtColor(cv2.imread("/media/samuel/HDD1TB/data.csail.mit.edu/graphics/fivek/img/tiff16_d/a0001-jmac_DSC1459.tif", cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
#expertE_RGB = cv2.cvtColor(cv2.imread("/media/samuel/HDD1TB/data.csail.mit.edu/graphics/fivek/img/tiff16_e/a0001-jmac_DSC1459.tif", cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)

In [ ]:
def basic_showImg(img, size=4):
    plt.figure(figsize=(size,size))
    plt.imshow(img)
    plt.show()

In [ ]:
basic_showImg(raw_RGB)
basic_showImg(expertA_RGB)

In [ ]:
raw_RGB = long_edge_resize500(raw_RGB)
expertA_RGB = long_edge_resize500(expertA_RGB)

In [ ]:
def sampled_luminance(rawRGB, editRGB, n_samples=51):
    rawLAB = cv2.cvtColor(rawRGB, cv2.COLOR_RGB2Lab)
    editLAB = cv2.cvtColor(editRGB, cv2.COLOR_RGB2Lab)
    raw_lum = rawLAB[:,:,0].flatten()/255*100
    edit_lum = editLAB[:,:,0].flatten()/255*100
    if raw_lum.shape <= edit_lum.shape :
        idx = np.random.randint(0,raw_lum.shape, n_samples)
    else : idx = np.random.randint(0,edit_lum.shape, n_samples)
        
    raw_sampled = raw_lum[idx]
    edit_sampled = edit_lum[idx]
    return raw_sampled, edit_sampled

In [ ]:
sampled_luminance(raw_RGB, expertA_RGB)

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
raw_sampled, edit_sampled = sampled_luminance(raw_RGB, expertA_RGB)
raw_sampled = raw_sampled.reshape(1, -1)
edit_sampled = edit_sampled.reshape(1, -1)

reg = GaussianProcessRegressor()

In [ ]:
reg.fit(raw_sampled, edit_sampled)

In [ ]:
reg.predict(raw_sampled)